In [1]:
import copy

In [162]:
class TweetsData:
    def __init__(self, username, tweet_text, likes, sentiment, tweet_date, date_descriptor):
        self.username = username
        self.tweet_text = tweet_text
        self.likes = likes
        self.sentiment = sentiment
        self.tweet_date = tweet_date
        self.date_descriptor = date_descriptor

    def __ge__(self, tweets_data):
        return self.likes >= tweets_data.likes

    def asdict(self):
        ret = {}
        ret['username'] = self.username
        ret['tweet_text'] = self.tweet_text
        ret['likes'] = self.likes
        ret['sentiment'] = self.sentiment
        ret['tweet_date'] = self.tweet_date
        ret['date_descriptor'] = self.date_descriptor
        return ret

class Candidate:
    def __init__(self, candidate):
        self.candidate = candidate
        self.pos = None
        self.neg = None
        self.combined = None

    def insert_polarity(self, candidate_polarity):
        if candidate_polarity.polarity == 'P':
            if self.pos is None:
                self.pos = candidate_polarity
            else:
                self.pos = self.pos.combine(candidate_polarity)
        else:
            if self.neg is None:
                self.neg = candidate_polarity
            else:
                self.neg = self.neg.combine(candidate_polarity)

    def add_pos(self, pos):
        self.pos = pos

    def add_neg(self, neg):
        self.neg = neg
        
    def combine_pos_neg(self):
        self.combined = self.pos.combine(self.neg) if self.pos is not None else copy.deepcopy(self.neg)

    def combine(self, candidate):
        c = self.candidate if self.candidate == candidate.candidate else None
        new_candidate = Candidate(c)
        pos = self.pos.combine(candidate.pos) if self.pos is not None else copy.deepcopy(candidate.pos)
        neg = self.neg.combine(candidate.neg) if self.neg is not None else copy.deepcopy(candidate.neg)
        new_candidate.add_pos(pos)
        new_candidate.add_neg(neg)
        return new_candidate

    def asdict(self):
        ret = {}
        ret['P'] = self.pos.asdict() if self.pos is not None else 'null'
        ret['N'] = self.neg.asdict() if self.neg is not None else 'null'
        ret['combined'] = self.combined.asdict() if self.combined is not None else 'null'
        return ret

class CandidatePolarity:
    def __init__(self, polarity, total_likes, num_tweets, avg_sentiment, tweet):
        self.polarity = polarity
        self.total_likes = total_likes
        self.num_tweets = num_tweets
        self.avg_sentiment = avg_sentiment
        self.tweet = tweet

    def combine(self, candidate_polarity):
        if candidate_polarity is None:
            return copy.deepcopy(self)
        polarity = self.polarity if self.polarity == candidate_polarity.polarity else None
        total_likes = self.total_likes + candidate_polarity.total_likes
        num_tweets = self.num_tweets + candidate_polarity.num_tweets
        avg_sentiment = (self.avg_sentiment * self.num_tweets + candidate_polarity.avg_sentiment * candidate_polarity.num_tweets) / num_tweets
        tweet = self.tweet if self.tweet >= candidate_polarity.tweet else candidate_polarity.tweet
        return CandidatePolarity(polarity, total_likes, num_tweets, avg_sentiment, tweet)

    def asdict(self):
        ret = {}
        ret['polarity'] = self.polarity
        ret['total_likes'] = self.total_likes
        ret['avg_sentiment'] = self.avg_sentiment
        ret['tweet'] = self.tweet.asdict()
        return ret

class Party:
    def __init__(self, party):
        self.party = party
        self.candidates = []
        self.combined = None

    def add_candidate(self, candidate):
        self.candidates.append(candidate)

    def combine_candidates(self):
        if len(self.candidates) == 0:
            return
        self.combined = self.candidates[0]
        for i in range(1, len(self.candidates)):
            self.combined = self.combined.combine(self.candidates[i])
        self.combined.combine_pos_neg()

    def asdict(self):
        ret = {}
        for c in self.candidates:
            ret[c.candidate] = c.asdict()
        ret['combined'] = self.combined.asdict()
        return ret
    
    

In [123]:
import random
def generate_random_p_dict(p):
    return {
    'polarity': p,
    'total_likes': 100,
    'num_tweets': 30,
    'avg_sentiment': 0.7 if p == 'P' else -0.6,
    # tweet
    'username': 'jmather25' if p == 'P' else 'Neg boy',
    'tweet_text': 'Happy bday',
    'likes': 80 if p == 'P' else 40,
    'sentiment': 0.9 if p == 'P' else -0.6,
    'tweet_date': 'May 26',
    'date_descriptor': 'exact'
    }

In [124]:
party = 'Democrat'
candidates = ['Bernie', 'Biden']

In [163]:
res = {}
for c in candidates:
    for p in ['P', 'N']:
        pk = generate_random_p_dict(p)
        td = None
        cp = None
        if pk is not None:
            td = TweetsData(pk['username'], pk['tweet_text'], pk['likes'], pk['sentiment'], pk['tweet_date'], pk['date_descriptor'])
            cp = CandidatePolarity(pk['polarity'], pk['total_likes'], pk['num_tweets'], pk['avg_sentiment'], td)
        cand = None
        if c in res:
            cand = res[c]
        else:
            cand = Candidate(c)
            res[c] = cand
        cand.insert_polarity(cp)
    
    

In [164]:
res['Bernie'].combine_pos_neg()

In [165]:
res['Bernie'].asdict()

{'P': {'polarity': 'P',
  'total_likes': 100,
  'avg_sentiment': 0.7,
  'tweet': {'username': 'jmather25',
   'tweet_text': 'Happy bday',
   'likes': 80,
   'sentiment': 0.9,
   'tweet_date': 'May 26',
   'date_descriptor': 'exact'}},
 'N': {'polarity': 'N',
  'total_likes': 100,
  'avg_sentiment': -0.6,
  'tweet': {'username': 'Neg boy',
   'tweet_text': 'Happy bday',
   'likes': 40,
   'sentiment': -0.6,
   'tweet_date': 'May 26',
   'date_descriptor': 'exact'}},
 'combined': {'polarity': None,
  'total_likes': 200,
  'avg_sentiment': 0.05,
  'tweet': {'username': 'jmather25',
   'tweet_text': 'Happy bday',
   'likes': 80,
   'sentiment': 0.9,
   'tweet_date': 'May 26',
   'date_descriptor': 'exact'}}}

In [148]:
res

{'Bernie': <__main__.Candidate at 0x102e90fd0>,
 'Biden': <__main__.Candidate at 0x102e604e0>}

In [166]:
py = Party('Democrat')
for c in candidates:
    res[c].combine_pos_neg()
    py.add_candidate(res[c])

In [157]:
py.candidates

[<__main__.Candidate at 0x104da6048>, <__main__.Candidate at 0x104da6780>]

In [167]:
py.combine_candidates()

In [168]:
import pprint
pprint.pprint(py.asdict())

{'Bernie': {'N': {'avg_sentiment': -0.6,
                  'polarity': 'N',
                  'total_likes': 100,
                  'tweet': {'date_descriptor': 'exact',
                            'likes': 40,
                            'sentiment': -0.6,
                            'tweet_date': 'May 26',
                            'tweet_text': 'Happy bday',
                            'username': 'Neg boy'}},
            'P': {'avg_sentiment': 0.7,
                  'polarity': 'P',
                  'total_likes': 100,
                  'tweet': {'date_descriptor': 'exact',
                            'likes': 80,
                            'sentiment': 0.9,
                            'tweet_date': 'May 26',
                            'tweet_text': 'Happy bday',
                            'username': 'jmather25'}},
            'combined': {'avg_sentiment': 0.05,
                         'polarity': None,
                         'total_likes': 200,
                         '